# 바이럴 지수 EDA: 단순 기사수 vs 바이럴 지수

단순 기사 카운트로는 볼 수 없었던 **트렌드의 역동성**을 바이럴 지수로 포착

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 데이터 로드
viral_index = pd.read_csv('../월별 바이럴 지수/2025_viral_index_matrix.csv', index_col=0)
news_monthly = pd.read_csv('correlation/news_monthly_by_category.csv', index_col=0, encoding='utf-8-sig')

# 미분류 제외
if '미분류' in news_monthly.columns:
    news_monthly = news_monthly.drop(columns=['미분류'])

# 공통 카테고리
categories = sorted(set(viral_index.columns) & set(news_monthly.columns))
months = viral_index.index.tolist()

print(f"분석 기간: {months[0]} ~ {months[-1]}")
print(f"카테고리: {len(categories)}개")

분석 기간: 2025-01 ~ 2025-12
카테고리: 10개


## 1. 순위 변동 차트 (Bump Chart): 단순 기사수 vs 바이럴 지수

**포인트**: 기사수 순위는 거의 고정, 바이럴 지수 순위는 역동적으로 변화

In [2]:
# 월별 순위 계산
count_rank = news_monthly[categories].rank(axis=1, ascending=False, method='min')
viral_rank = viral_index[categories].rank(axis=1, ascending=False, method='min')

# Bump Chart 데이터 준비
def prepare_bump_data(rank_df, label):
    data = []
    for cat in categories:
        for month in rank_df.index:
            data.append({
                '월': month,
                '카테고리': cat,
                '순위': rank_df.loc[month, cat],
                '지표': label
            })
    return pd.DataFrame(data)

bump_count = prepare_bump_data(count_rank, '단순 기사수')
bump_viral = prepare_bump_data(viral_rank, '바이럴 지수')

# 서브플롯으로 비교
fig = make_subplots(rows=1, cols=2, subplot_titles=['단순 기사수 순위', '바이럴 지수 순위'],
                    horizontal_spacing=0.1)

colors = px.colors.qualitative.Set2

for i, cat in enumerate(categories):
    # 기사수 순위
    cat_data = bump_count[bump_count['카테고리'] == cat]
    fig.add_trace(go.Scatter(
        x=cat_data['월'], y=cat_data['순위'],
        mode='lines+markers', name=cat,
        line=dict(color=colors[i % len(colors)], width=2),
        marker=dict(size=8),
        legendgroup=cat, showlegend=True
    ), row=1, col=1)
    
    # 바이럴 지수 순위
    cat_data = bump_viral[bump_viral['카테고리'] == cat]
    fig.add_trace(go.Scatter(
        x=cat_data['월'], y=cat_data['순위'],
        mode='lines+markers', name=cat,
        line=dict(color=colors[i % len(colors)], width=2),
        marker=dict(size=8),
        legendgroup=cat, showlegend=False
    ), row=1, col=2)

fig.update_yaxes(autorange='reversed', title='순위', row=1, col=1)
fig.update_yaxes(autorange='reversed', title='순위', row=1, col=2)
fig.update_xaxes(tickangle=45)
fig.update_layout(
    height=500, 
    title='<b>순위 변동 비교</b>: 기사수는 고정적, 바이럴 지수는 역동적',
    legend=dict(orientation='h', yanchor='bottom', y=-0.3, x=0.5, xanchor='center')
)
fig.show()

## 2. 바이럴 지수 히트맵

**포인트**: 어느 카테고리가 어느 시점에 "핫"했는지 한눈에 파악

In [3]:
# 히트맵용 데이터
heatmap_data = viral_index[categories].T

fig = go.Figure(data=go.Heatmap(
    z=heatmap_data.values,
    x=heatmap_data.columns,
    y=heatmap_data.index,
    colorscale='RdYlBu_r',
    zmid=0,
    text=np.round(heatmap_data.values, 1),
    texttemplate='%{text}',
    textfont=dict(size=9),
    hovertemplate='%{y}<br>%{x}<br>바이럴 지수: %{z:.1f}<extra></extra>'
))

fig.update_layout(
    title='<b>바이럴 지수 히트맵</b>: 빨강=급등, 파랑=급락',
    xaxis_title='월',
    yaxis_title='카테고리',
    height=450,
    xaxis=dict(tickangle=45)
)
fig.show()

## 2-1. 카테고리별 월별 바이럴 지수 순위

**포인트**: 어느 카테고리가 몇 위였는지 순위 변동 확인

In [4]:
# 바이럴 지수 순위 계산
viral_rank = viral_index[categories].rank(axis=1, ascending=False, method='min').astype(int)
rank_data = viral_rank.T

# 순위 히트맵
fig = go.Figure(data=go.Heatmap(
    z=rank_data.values,
    x=rank_data.columns,
    y=rank_data.index,
    colorscale='RdYlGn_r',  # 1위=초록, 10위=빨강
    zmin=1, zmax=10,
    text=rank_data.values,
    texttemplate='%{text}위',
    textfont=dict(size=10),
    hovertemplate='%{y}<br>%{x}<br>순위: %{z}위<extra></extra>'
))

fig.update_layout(
    title='<b>카테고리별 월별 바이럴 지수 순위</b>: 초록=상위, 빨강=하위',
    xaxis_title='월',
    yaxis_title='카테고리',
    height=450,
    xaxis=dict(tickangle=45)
)
fig.show()

# 테이블로도 출력
print("\n[바이럴 지수 순위 테이블]")
viral_rank.T


[바이럴 지수 순위 테이블]


,2025-01,2025-02,2025-03,2025-04,2025-05,2025-06,2025-07,2025-08,2025-09,2025-10,2025-11,2025-12
거시경제/금융정책,1,10,8,3,7,2,10,7,7,7,6,3
경제이론/학술,10,1,10,2,3,3,2,10,1,9,10,2
금융시스템/위기,6,9,4,4,2,6,9,9,3,5,7,4
기업경영/리더십,2,6,6,7,6,10,8,2,6,6,1,8
부동산/실물자산,8,5,5,6,5,4,4,5,2,3,8,6
재테크/개인금융,7,2,7,9,1,8,1,8,9,8,3,7
주식투자/트레이딩,9,8,2,5,8,5,7,6,5,4,4,9
지정학/국제정세,4,4,3,1,9,9,3,4,8,2,9,10
테크/스타트업,5,7,9,8,4,7,5,3,4,1,5,5
투자철학/대가,3,3,1,10,10,1,6,1,10,10,2,1


## 3. 단순 기사수 vs 바이럴 지수 트렌드 비교

**포인트**: 기사수는 완만, 바이럴 지수는 변동성이 큼 → 트렌드 "변화"를 포착

In [5]:
# 드롭다운으로 카테고리 선택
fig = go.Figure()

buttons = []
for i, cat in enumerate(categories):
    # 기사수 (정규화)
    count_norm = (news_monthly[cat] - news_monthly[cat].mean()) / news_monthly[cat].std()
    viral_vals = viral_index[cat]
    
    visible = (i == 0)
    
    fig.add_trace(go.Scatter(
        x=months, y=count_norm,
        name='기사수 (Z-Score)', mode='lines+markers',
        line=dict(color='gray', width=2, dash='dot'),
        visible=visible
    ))
    fig.add_trace(go.Scatter(
        x=months, y=viral_vals,
        name='바이럴 지수', mode='lines+markers',
        line=dict(color='crimson', width=3),
        visible=visible
    ))
    
    vis_list = [False] * (len(categories) * 2)
    vis_list[i*2:i*2+2] = [True, True]
    buttons.append(dict(label=cat, method='update', args=[{'visible': vis_list}]))

fig.update_layout(
    updatemenus=[dict(buttons=buttons, direction='down', x=0, y=1.15, showactive=True)],
    title='<b>기사수 vs 바이럴 지수</b>: 바이럴 지수가 변화를 더 민감하게 포착',
    xaxis_title='월', yaxis_title='지수',
    height=450,
    legend=dict(orientation='h', y=-0.15),
    xaxis=dict(tickangle=45)
)
fig.add_hline(y=0, line_dash='dash', line_color='black', opacity=0.3)
fig.show()

## 4. 변동성 비교: 기사수 vs 바이럴 지수

**포인트**: 바이럴 지수가 카테고리 간 변동성을 더 균등하게 만들어 비교 가능하게 함

In [6]:
# 변동성 (표준편차) 계산
count_std = news_monthly[categories].std()
viral_std = viral_index[categories].std()

# 변동계수 (CV) - 평균 대비 변동성
count_cv = news_monthly[categories].std() / news_monthly[categories].mean() * 100
viral_cv = viral_index[categories].std()  # 바이럴 지수는 이미 변화율 기반

fig = make_subplots(rows=1, cols=2, 
                    subplot_titles=['기사수 변동계수 (CV%)', '바이럴 지수 표준편차'])

# 기사수 CV
fig.add_trace(go.Bar(
    x=categories, y=count_cv.values,
    marker_color='steelblue', name='기사수 CV'
), row=1, col=1)

# 바이럴 지수 표준편차
fig.add_trace(go.Bar(
    x=categories, y=viral_std.values,
    marker_color='crimson', name='바이럴 지수 Std'
), row=1, col=2)

fig.update_layout(
    height=400,
    title='<b>변동성 비교</b>: 바이럴 지수는 카테고리 간 비교가 용이',
    showlegend=False
)
fig.update_xaxes(tickangle=45)
fig.show()

## 요약

| 시각화 | 인사이트 |
|--------|----------|
| **순위 변동 차트** | 기사수 순위는 고정적 (주식 항상 1위), 바이럴 지수는 매월 역동적 변화 |
| **히트맵** | 언제 어떤 카테고리가 "핫"했는지 한눈에 파악 가능 |
| **트렌드 비교** | 기사수는 완만한 변화, 바이럴 지수는 급등/급락 민감하게 포착 |
| **변동성 비교** | 바이럴 지수는 카테고리 간 스케일 차이를 보정하여 비교 가능 |